In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_csv('/Users/soham2312/Downloads/UpdatedResumeDataSet.csv')
# print(data['Category'].value_counts())

import re

def clean(text):
    text=re.sub('http\S+\s*', ' ', text)
    text=re.sub('RT|cc', ' ', text)
    text=re.sub('#\S+', '', text)
    text=re.sub('@\S+', '', text)
    text=re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)
    text=re.sub('\s+', ' ', text)
    text=re.sub(r'[^\x00-\x7f]', r' ', text)
    return text

data['clean text']=data.Resume.apply(lambda x: clean(x))
# data.drop('Resume', inplace=True, axis=1)


import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

stopwords=set(stopwords.words('english')+['``',"''"])

total_words=[]
sentences=data['clean text'].values
cleanSentences =""

for i in range(0,962):
    text=clean(sentences[i])
    cleanSentences+=text
    words=nltk.word_tokenize(text)
    for word in words:
        if word not in stopwords and word not in string.punctuation:
            total_words.append(word)
            
word_freq_dist=nltk.FreqDist(total_words)
most_common=word_freq_dist.most_common(100)

def stem(text):
    t=[]
    for i in text.split():
        t.append(ps.stem(i))
    return " ".join(t)
data['clean text']=data['clean text'].apply(stem)
data['clean text']=data['clean text'].apply(lambda x:x.lower())

s=[]
for i in range(len(most_common)):
    s.append(most_common[i][0])
s=[" ".join([stem(word) for word in sentence.split(" ")]) for sentence in s]


from sklearn.preprocessing import LabelEncoder

var=['Category']
le=LabelEncoder()

for i in var:
    data[i]=le.fit_transform(data[i])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

text=data['clean text'].values

terget=data['Category'].values
# print(s)
# print(text)
vect=TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=4000)

vect.fit(text)


Word_feature=vect.transform(text)

x_train, x_test, y_train, y_test=train_test_split(Word_feature, terget, random_state=0, test_size=0.2)

import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

model=OneVsRestClassifier(KNeighborsClassifier())
model.fit(x_train, y_train)

prediction=model.predict(x_test)
from sklearn import metrics
print("model report: %s: \n %s\n" % (model, metrics.classification_report(y_test, prediction)))
print(prediction)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soham2312/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/soham2312/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


model report: OneVsRestClassifier(estimator=KNeighborsClassifier()): 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00         9
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         5
           6       0.82      1.00      0.90         9
           7       1.00      0.71      0.83         7
           8       1.00      0.91      0.95        11
           9       1.00      1.00      1.00         9
          10       1.00      1.00      1.00         8
          11       0.90      1.00      0.95         9
          12       1.00      1.00      1.00         5
          13       1.00      1.00      1.00         9
          14       1.00      1.00      1.00         7
          15       1.00      1.00      1.00        19
          